In [7]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

df = pd.read_csv("data/cleaned_data.csv")

df = pd.get_dummies(df, columns=["region", "gender"], drop_first=True)

feature_cols = ["age", "income_per_age", "log_expenses"] + [
    col for col in df.columns if col.startswith("region_") or col.startswith("gender_")
]
X = df[feature_cols]
y = df["income"]

X = X.apply(pd.to_numeric, errors="coerce")
y = pd.to_numeric(y, errors="coerce")

# Convert bools to ints
X = X.astype({col: int for col in X.select_dtypes(include=["bool"]).columns})

# Drop rows with any NaNs
mask = X.notnull().all(axis=1) & y.notnull()
X = X[mask]
y = y[mask]

X = sm.add_constant(X)

assert all([np.issubdtype(dt, np.number) for dt in X.dtypes]), "Non-numeric found in X"
assert np.issubdtype(y.dtype, np.number), "Non-numeric found in y"

model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     31.20
Date:                Mon, 04 Aug 2025   Prob (F-statistic):           1.42e-42
Time:                        22:06:07   Log-Likelihood:                -5044.9
No. Observations:                 470   AIC:                         1.011e+04
Df Residuals:                     460   BIC:                         1.015e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.903e+04   1.42e+04      2.